In [ ]:
pip install cssselect

In [89]:
import numpy as np 
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import csv
from lxml import html
import requests
import pandas as pd
import numpy as np
import re

**Teams' pages scraping**

In this section, we are scraping the pages for each team in Primier League. To do so , we will first access the page containing all clubs. Then , we will loop through all teams and save their links in a list so we can use them latter. In the clubs page, each team is represented by a link that takes you to the team page containing its stats and overview

In [90]:
#Take site and structure html

page = requests.get('https://www.premierleague.com/clubs')
tree = html.fromstring(page.content)

In [91]:
# extract link locations related to the teams contained in the page
linkLocation = tree.cssselect('.indexItem')

In [92]:
#Create an empty list for us to send each team's link to
teamLinks = []

# There are 20 teams, we will loop through them 
for i in range(0,20):
    
    # A variable that contains the ith team's link location
    temp = linkLocation[i].attrib['href']
    
    # To access the page of each team using the code i.e not manually, we will add the team's link to the website domain 
    temp = "http://www.premierleague.com/" + temp
    print(temp)
    # When opening the team's page it will be the overview page, we will change it to squad since we are interested in players
    temp = temp.replace("overview", "squad")
   
    #Add the finished link to our teamLinks list
    teamLinks.append(temp)

http://www.premierleague.com//clubs/1/Arsenal/overview
http://www.premierleague.com//clubs/2/Aston-Villa/overview
http://www.premierleague.com//clubs/131/Brighton-and-Hove-Albion/overview
http://www.premierleague.com//clubs/43/Burnley/overview
http://www.premierleague.com//clubs/4/Chelsea/overview
http://www.premierleague.com//clubs/6/Crystal-Palace/overview
http://www.premierleague.com//clubs/7/Everton/overview
http://www.premierleague.com//clubs/34/Fulham/overview
http://www.premierleague.com//clubs/9/Leeds-United/overview
http://www.premierleague.com//clubs/26/Leicester-City/overview
http://www.premierleague.com//clubs/10/Liverpool/overview
http://www.premierleague.com//clubs/11/Manchester-City/overview
http://www.premierleague.com//clubs/12/Manchester-United/overview
http://www.premierleague.com//clubs/23/Newcastle-United/overview
http://www.premierleague.com//clubs/18/Sheffield-United/overview
http://www.premierleague.com//clubs/20/Southampton/overview
http://www.premierleague.com

**Players' pages scraping**

In this section, we will use the links we have got for each team and we will make a list of similar links for all playeras in that team. This will help us scrape the information of each player later  

In [93]:
#Create empty lists for player links
#First list is used for links in the overview page of each player while the second is used for the stats page
#Each page contains information we will need for each player
playerLink1 = []
playerLink2 = []

#looping through teams using their links
for i in range(len(teamLinks)):
    
    #accessing each team page 
    squadPage = requests.get(teamLinks[i])
    squadTree = html.fromstring(squadPage.content)
    
    #Extracting players' links, they are under class 'playerOverviewCard'
    playerLocation = squadTree.cssselect('.playerOverviewCard')

    #loop through players links in order to add them to the website domain. This eliminates the need for accessing each players page manually
    for i in range(len(playerLocation)):
        
        # add the link to the website domain 
        playerLink1.append("http://www.premierleague.com/" + playerLocation[i].attrib['href'])
       
        #...For the second link, change the page from player overview to stats
        #playerLink2.append(playerLink1[i])
       

In [95]:
for i in range(len(playerLink1)):
    temp=playerLink1[i].replace("overview", "stats")
    playerLink2.append(playerLink1[i].replace("overview", "stats"))

In [96]:
playerLink1[50]

'http://www.premierleague.com//players/51423/John-McGinn/overview'

In [97]:
playerLink2[50]

'http://www.premierleague.com//players/51423/John-McGinn/stats'

In [98]:
#Create lists for each variable
Name = []
Team = []
Age = []
Apps = []
HeightCM = []
WeightKG = []
Goals= []
Assists=[]
Passes=[]
Wins=[]
Losses=[]
Yellowcards=[]
Redcards=[]
Fouls=[]
Position=[]

In [99]:
for i in range(len(playerLink1)):

    #...download and process the two pages collected earlier...
    playerPage1 = requests.get(playerLink1[i])
    playerTree1 = html.fromstring(playerPage1.content)
    playerPage2 = requests.get(playerLink2[i])
    playerTree2 = html.fromstring(playerPage2.content)

    #...find the relevant datapoint for each player, starting with name...
    tempName = str(playerTree1.cssselect('div.name')[0].text_content())
    #print(tempName)
    
    #...and team, but if there isn't a team, return "BLANK"...
    try:
        tempTeam = str(playerTree1.cssselect('.table:nth-child(1) .long')[0].text_content())
    except IndexError:
        tempTeam = str("BLANK")
    #print(tempTeam)
    
    #...and age, but if this isn't there, leave a blank 'no number' number...
    try:  
        tempAge = playerTree1.cssselect('.pdcol2 li:nth-child(1) .info--light')[0].text_content()
        res= tempAge[tempAge.find("(")+1:tempAge.find(")")]
        #res = re.findall(r"\(\s*\+?(-?\d+)\s*\)", tempAge)
    except IndexError:
        tempAge = float('NaN')
    #print(res) 

    #...and appearances. This is a bit of a mess on the page, so tidy it first...
    try:
        tempApps = playerTree2.cssselect('.statappearances')[0].text_content()
        tempApps = int(re.search(r'\d+', tempApps).group())
    except IndexError:
        tempApps = float('NaN')
        

    #...and height. Needs tidying again...
    try:
        tempHeight = playerTree1.cssselect('.pdcol3 li:nth-child(1) .info')[0].text_content()
        tempHeight = int(re.search(r'\d+', tempHeight).group())
    except IndexError:
        tempHeight = float('NaN')

    #...and weight. Same with tidying and returning blanks if it isn't there
    try:
        tempWeight = playerTree1.cssselect('.pdcol3 li+ li .info')[0].text_content()
        tempWeight = int(re.search(r'\d+', tempWeight).group())
    except IndexError:
        tempWeight = float('NaN')
    
    try:
        tempgoals =  playerTree2.cssselect('.statgoals')[0].text_content()
        tempgoals = int(re.search(r'\d+', tempgoals).group())
    except IndexError:
        tempgoals = float('NaN')
    #print(tempgoals)
    
    try:
        tempassists =  playerTree2.cssselect('.statgoal_assist')[0].text_content()
        tempassists = int(re.search(r'\d+', tempassists).group())
    except IndexError:
        tempassists = float('NaN')
    #print(tempassists)
    
    try:
        tempapasses =  playerTree2.cssselect('.stattotal_pass')[0].text_content()
        res2=tempapasses.replace(",", "")
        tempapasses = int(re.search(r'\d+', res2).group())
        
    except IndexError:
        tempapasses = float('NaN')
    #print(tempapasses)
    
    
    try:
        tempwins = playerTree2.cssselect('.statwins')[0].text_content()
        tempwins = int(re.search(r'\d+', tempwins).group())
    except IndexError:
        tempwins = float('NaN')
    
    try:
        templosses = playerTree2.cssselect('.statlosses')[0].text_content()
        templosses = int(re.search(r'\d+', templosses).group())
    except IndexError:
        templosses = float('NaN')
        
    try:
        tempYellowcards = playerTree2.cssselect('.statyellow_card')[0].text_content()
        tempYellowcards = int(re.search(r'\d+', tempYellowcards).group())
    except IndexError:
        tempYellowcards = float('NaN')
        
    try:
        tempRedcards = playerTree2.cssselect('.statred_card')[0].text_content()
        tempRedcards = int(re.search(r'\d+', tempRedcards).group())
    except IndexError:
        tempRedcards = float('NaN')
        
    try:
        tempFouls = playerTree2.cssselect('.statfouls')[0].text_content()
        tempFouls = int(re.search(r'\d+', tempFouls).group())
    except IndexError:
        tempFouls = float('NaN')     
        
    try:
        tempposition = str(playerTree1.cssselect('.playerOverviewAside .info')[1].text_content())
    except IndexError:
        tempposition = str("BLANK")
    #print(tempposition)
    
    #Now that we have a player's full details - add them all to the lists
    
    Name.append(tempName)
    Team.append(tempTeam)
    Age.append(res)
    Apps.append(tempApps)
    HeightCM.append(tempHeight)
    WeightKG.append(tempWeight)
    Goals.append(tempgoals)
    Assists.append(tempassists)
    Passes.append(tempapasses)
    Wins.append(tempwins)
    Losses.append(templosses)
    Yellowcards.append(tempYellowcards)
    Redcards.append(tempRedcards)
    Fouls.append(tempFouls)
    Position.append(tempposition)

In [101]:
#Create data frame from lists
df = pd.DataFrame(
    {'Name':Name,
     'Team':Team,
     'Positon':Position,
     'Age':Age,
     'Apps':Apps,
     'HeightCM':HeightCM,
     'WeightKG':WeightKG,
    'Goals':Goals,
    'Assists':Assists,
    'Passes':Passes,
     'Wins':Wins,
      'Losses':Losses,
   'Yellowcards':Yellowcards,
    'Redcards':Redcards,
    'Fouls':Fouls})

#Show me the top 3 rows:

df.head(50)

,Name,Team,Positon,Age,Apps,HeightCM,WeightKG,Goals,Assists,Passes,Wins,Losses,Yellowcards,Redcards,Fouls
0,Bernd Leno,Arsenal,Goalkeeper,29,93,190.0,83.0,0,0,2616,39,28,2,1,2
1,RÃºnar Alex RÃºnarsson,Arsenal,Goalkeeper,26,1,186.0,76.0,0,0,9,0,1,0,0,0
2,Karl Hein,Arsenal,Goalkeeper,19,0,193.0,NaN,0,0,0,0,0,0,0,0
3,Mat Ryan,Arsenal (Loan),Goalkeeper,29,124,184.0,82.0,0,0,3596,29,55,3,0,2
4,Arthur Okonkwo,Arsenal,Goalkeeper,19,0,NaN,NaN,0,0,0,0,0,0,0,0
5,James Hillson,Arsenal,Goalkeeper,20,0,NaN,NaN,0,0,0,0,0,0,0,0
6,HÃ©ctor BellerÃ­n,Arsenal,Defender,26,182,178.0,74.0,8,20,8136,98,47,30,0,143
7,Kieran Tierney,Arsenal,Defender,23,38,178.0,70.0,2,3,1528,15,15,6,0,19
8,Rob Holding,Arsenal,Defender,25,65,189.0,75.0,0,1,3600,33,20,10,0,49
9,Calum Chambers,Arsenal,Defender,26,145,182.0,66.0,6,3,6207,49,59,28,1,141


In [9]:
df.shape

(638, 15)

In [102]:
df[df['Team']=='Manchester City']

,Name,Team,Positon,Age,Apps,HeightCM,WeightKG,Goals,Assists,Passes,Wins,Losses,Yellowcards,Redcards,Fouls
360,Ederson,Manchester City,Goalkeeper,27,142,188.0,86.0,0,2,3612,110,18,9,1,5
361,Zack Steffen,Manchester City,Goalkeeper,26,1,191.0,86.0,0,0,26,1,0,0,0,1
364,Nathan AkÃ©,Manchester City,Defender,26,154,180.0,75.0,13,6,6154,55,69,19,1,102
365,Kyle Walker,Manchester City,Defender,30,314,178.0,70.0,8,30,17157,196,53,46,1,263
366,John Stones,Manchester City,Defender,26,182,188.0,70.0,5,0,10482,106,41,7,1,51
367,Oleksandr Zinchenko,Manchester City,Defender,24,58,175.0,64.0,0,3,4297,48,8,2,1,43
368,Aymeric Laporte,Manchester City,Defender,26,74,191.0,86.0,4,3,6132,62,7,6,0,48
369,Benjamin Mendy,Manchester City,Defender,26,47,185.0,85.0,2,9,2437,35,5,8,0,41
370,JoÃ£o Cancelo,Manchester City,Defender,26,42,182.0,74.0,1,3,2722,30,7,8,0,50
372,RÃºben Dias,Manchester City,Defender,23,29,186.0,76.0,1,0,2496,22,2,3,0,19
